# Assignment 3: Universal Function Approximator


The goal of this exercise is to train a two-layer fully-connected network to perform one-dimensional non-linear regression via gradient descent. To show the flexibility of the approach, three different functions will be approximated. First, the network and its gradient need to be implemented. Second, target data for three different functions will be generated. Finally, the training procedure will be applied to the data, and the resulting approximated function will be plotted together with the data samples.

## Network Implementation

A two-layer network is defined by parameters $\Theta=(\mathbf W^{(1)},\vec w^{(2)})$ that are split into $\mathbf W^{(1)}\in\mathbb R^{K\times {(D+1)}}$ for the first layer and $\vec w^{(2)}\in\mathbb R^{K+1}$ for the second layer. In our case, since we have only a single input, we have $D=1$.
For a given input $\vec x = (1, x)^T$, the network is implemented in three steps:

1. Compute the first layer output, aka, the activation: $\vec a_- = \mathbf W^{(1)} \vec x$
2. Apply the activation function for each element of $\vec a_- : \vec h_- = g(\vec a_-)$ and prepend the bias neuron $h_0=1$ to arrive at $\vec h$.
3. Compute the output of the network: $y = \vec w^{(2)}\ ^T\vec h$.

### Task 1  
Implement a function that returns the network output for a given input $\vec x$ and parameters $\Theta=(\mathbf W^{(1)}, \vec w^{(2)})$. Remember that the input of the function $\vec x = (1, x)^T$. Also remember to prepend $h_0=1$ in your implementation.

We use hyperbolic tangent $(\tanh)$ as the activation function:

\begin{equation*}
    \tanh(a) = \frac{e^{a}-e^{-a}}{e^{a}+e^{-a}}
\end{equation*}

Note:

1. Use the `np` implemention of the hyperbolic tangent function.
2. Use `np.concatenate` or `np.insert` to prepend $h_0$.
3. Make use of `np.dot` to compute matrix-vector and vector-vector products.

In [128]:
import numpy as np
import math

def network(x, Theta):
  W1, w2 = Theta
  # activation function
  a_ = np.dot(W1, x)
  h_ = np.tanh(a_)
  h = np.array([1])
  y = np.dot(w2.T, np.append(h, h_))
  return y, h

Test 1: Sanity Check
----------------------------

We select a specific number of hidden neurons and create the weights accordingly, using all zeros in the first layer and all ones in the second. The test case below assures that the function from Task 1 actually returns $1$ for those weights.

Note: your function should pass the test below.

In [129]:
K = 20
D = 1
W1 = np.zeros((K, D+1))
w2 = np.ones(K+1)
x = np.random.rand(D+1)

y, _ = network(x, (W1, w2))
assert abs(1 - y) < 1e-6
print("Test passed")

Test passed


## Gradient Implementation

In order to perform gradient descent, we need to define a loss function. As provided in the lecture, the $\mathcal J^{L_2}$ loss function is defined over a dataset $X=\{(\vec x^{[n]}, t^{[n]})\}$, that is defined as a list of tuples, as follows:

$$
   \mathcal J^{L_2} = \frac{1}{N}\sum_{i=1}^N (y^{[n]}-t^{[n]})^2
$$

where $y^{[n]}$ is the output of the network from Task 1 when inputting $\vec x^{[n]}$. Interestingly, however, we never explicitly need to compute the output of the loss function. It is only used to analytically compute the gradient as shown in the lecture.

The gradient is composed of two items, one for each layer. Particularly, for a given dataset $X$, the gradient of loss $J^{L_2}$ is defined as:

\begin{align}
  \frac{\partial \mathcal J}{\partial w_{kd}^{(1)}} &= \frac{2}{N} \sum\limits_{n=1}^N (y^{[n]}-t^{[n]}) w_{k}^{(2)} (1-h_{k}^{[n]}\cdot h_{k}^{[n]}) x_{d}^{[n]}\\
  \frac{\partial \mathcal J}{\partial w_{k}^{(2)}} &= \frac{2}{N} \sum\limits_{n=1}^N (y^{[n]}-t^{[n]}) w_{k}^{(2)} h_{k}^{[n]}
\end{align}

### Task 2
Implement a function that returns the gradient as defined in $(1)$ and $(2)$ for a given dataset $X$, and given weights $\Theta=(\mathbf W^{(1)},\vec w^{(2)})$. Make sure that both parts of the gradient are computed. 

Hint:

1. Make use of the the function implemented in Task 1 where appropriate

Note:

  1. This is a slow implementation. We will see how to speed this up in the next lecture.
  2. You can make use of `np.zeros` to initialize the gradient.
  3. The outper product can be computed via `np.outer`.
  4. Remember that we used the $\tanh$ activation function in our network.

In [130]:
def gradient(X, Theta):
  # split parameters for easier handling
  W1, w2 = Theta
  
  # define gradient with respect to both parameters
  dW1 = np.zeros(W1.shape)
  dw2 = np.zeros(w2.shape)

  # iterate over dataset
  for x, t in X:
    # compute the gradient
    y, h = network(x, Theta)
    dW1.append((y - t) * w2[1:] * (1 - h**2) * x)
    dw2.append((y - t) * h * w2[1:]) 

  # anything else?
  dW1 = 2 * np.mean(dW1, axis=0)
  dw2 = 2 * np.mean(dw2, axis=0)
  
  return dW1, dw2

## Gradient Descent

The procedure of gradient descent is the repeated application of two steps.
 
1. The gradient of loss $\nabla_{\Theta}\mathcal J^{L_2}$ is computed based on the current value of the parameters $\Theta=(\mathbf W^{(1)},\vec w^{(2)})$.
2. The weights are updated by moving a small step $\eta$ into the direction of the negative gradient:

$$
    \Theta = \Theta - \eta \nabla_{\Theta}\mathcal J
$$

As stopping criterion, we select the number of training epochs to be 10000.

### Task 3
Implement a function that performs gradient descent for a given dataset $X$, given initial parameters $\Theta$ and a given learning rate $\eta$ and returns the optimized parameters $\Theta^*$.

In [131]:
def gradient_descent(X, Theta, eta):
  epochs = 10000
  # perform iterative gradient descent
  for epoch in range(epochs):
    # compute the gradient
    dW1, dw2 = gradient(X, Theta)

    # update the parameters
    Theta[0] = Theta[0] - eta * dW1
    Theta[1]  = Theta[1] - eta * dw2

  # return optimized parameters
  return Theta

## Generate Datasets

In total, we will test our gradient descent function with three different datasets. Particularly, we approximate:

1. $X_1: t = \sin(2x)$ for $x\in[-2,2]$
2. $X_2: t = e^{-2x^2}$ for $x\in[-2,2]$
3. $X_3: t = -x^5 - 3x^4 + 11x^3 + 27x^2 - 10x - 64$ for $x\in[-4.5,3.5]$

### Task 4

Generate dataset $X_1$, for $N=50$ samples randomly drawn from range $x\in[-2,2]$. 
Generate data $X_2$ for $N=30$ samples randomly drawn from range $x\in[-2,2]$. 
Generate dataset $X_3$ for $N=200$ samples randomly drawn from range $x\in[-4.5,3.5]$. 
Implement all three datasets as lists of tuples: $\{(\vec x^{[n]}, t^{[n]})\mid 1\leq n\leq N\}$.

Note:

  1. You can use `np.random.uniform` to create uniformly distributed samples for $x$.
  2. Make sure that $\vec x = (1, x)^T$ for each sample.
  3. You can make use of `np.sin`, `np.exp` and `np.pow` to compute target values.

In [141]:
# X1 = [((1, x), np.sin(2 * x_)) for x_ in np.random.uniform(low=-2.0, high=2.0, size=50)]
# X2 = [((1, x),  np.exp(-2 * x_ **2)) for x_ in np.random.uniform(low=-2.0, high=2.0, size=30)]
# X3 = [((1, x),  -x_ ** 5 - 3 * x_**4 + 11 * x_**3 + 27 * x_**2 - 10 * x_ - 64) for x_ in np.random.uniform(low=-4.5, high=3.5, size=200)]


     

X1 = [((1, x := np.random.uniform(-2, 2, 1)[0]), np.sin(2*x)) for i in range(50)]
X2 = [((1, x := np.random.uniform(-2, 2, 1)[0]), np.exp(-2*(x)**2)) for i in range(30)]
X3 = [((1, x := np.random.uniform(-4.5, 3.5, 1)[0]), -x**5 - 3*x**4 + 11*x**3 + 27*x**2 - 10*x - 64) for i in range(200)]



In [142]:
# x = np.random.rand(D+1)
X1

[((1, 0.0986961631492127), 0.1961129643790878),
 ((1, 1.8458553610429838), -0.5227878815755223),
 ((1, 1.5718746920121753), -0.0021567287625575216),
 ((1, -0.4412055085861102), -0.7722728196076941),
 ((1, -1.328424606589012), -0.46598137226692654),
 ((1, -1.820631418993143), 0.47913607211161807),
 ((1, 0.29997050893304333), 0.5645937923571364),
 ((1, 0.5093944248380349), 0.8514735222907335),
 ((1, 1.2223524705650926), 0.6418341622195038),
 ((1, 0.5641575829230332), 0.9036920550405912),
 ((1, 1.317707642468219), 0.4848375077521499),
 ((1, 1.9114902400571179), -0.6298715541685973),
 ((1, -0.40593858347604517), -0.725580201176684),
 ((1, 0.20828315199692726), 0.4046227855727036),
 ((1, 1.971183500815103), -0.7178953692520483),
 ((1, -1.027608206570736), -0.8849451609283395),
 ((1, -1.1159135490453407), -0.7893598286781274),
 ((1, -1.0762112704386437), -0.8355703400217888),
 ((1, -1.3579488125350272), -0.41295388000564365),
 ((1, 1.1015234063229622), 0.8066996012872051),
 ((1, 0.5790741963

### Test 2: Sanity Check

The test case below assures that the elements of each generated dataset are tuples with two elements, that the first element ($\vec x$) is a vector with two numbers, and that the second element ($t$) is a float.

In [143]:
assert all(
    isinstance(x, (tuple,list)) and 
    len(x) == 2 and 
    isinstance(x[0], (tuple,list,np.ndarray)) and 
    len(x[0] == 2) and 
    isinstance(x[1], float)
    for X in (X1, X2, X3)
    for x in X
)

print('Test passed!')

TypeError: object of type 'bool' has no len()

## Function Approximation

Finally, we want to make use of our gradient descent implementation to approximate our functions. In order to see our success, we want to plot the functions together with the data.

### Task 5 (theoretical question)

When looking at the example plots in the exercise slides (exemplary solutions for the plotting Task 8), how many hidden neurons $K$ do we need in order to approximate the functions? Is there any difference
1 between the three target functions?

In [ ]:
K1 = ...
K2 = ...
K3 = ...

### Task 6

For each of the datasets, randomly initialize the parameters $\Theta_1,\Theta_2,\Theta_3\in[-1,1]$ according to the number of hidden neurons estimated in Task 5.

Note:

  1. You can use `np.random.uniform` to initialize the weights.
  2. Make sure that the weight matrices are instantiated in the correct dimensions.

In [ ]:
Theta1 = ...
Theta2 = ...
Theta3 = ...

### Task 7

Call gradient descent function from Task 3 using the datasets $X_1, X_2, X_3$, the according created parameters $\Theta_1,\Theta_2,\Theta_3$ and a learning rate of $\eta=0.1$. Store the resulting optimized weights $\Theta_1^*, \Theta_2^*, \Theta_3^*$ and the loss values.

Optimize the learning rate $\eta$ for each of the three functions. Do you see any differences? What are the best learning rates that you can find?

WARNING: Depending on the implementation, this might run for several minutes!

In [ ]:
...
...
...

## Data and Function Plotting

### Task 8

Implement a plotting function that takes a given dataset $X$, given parameters $\Theta$ and a defined range $R$. Each data sample $(x^{[n]},t^{[n]})$ of the dataset is plotted as an $''x''$. In order to plot the function that is approximated by the network, generate sufficient equally-spaced input values $x\in R$, compute the network output $y$ for these inputs, and plot them with a line.

Note:

  1. The dataset $X$ is defined as above, a list of tuples $(\vec x, t)$.
  2. Each input in the dataset is defined as $\vec x = (1,x)^T$.
  3. Equidistant points can be obtained via `np.arange`.

In [ ]:
from matplotlib import pyplot
def plot(X, Theta, R):
  # first, plot data samples
  pyplot.plot(..., ..., "rx", label="Data")

  # define equidistant points from R[0] to R[1] to evaluate the network
  x = ...
  # compute the network outputs for these values
  y = ...
  # plot network approximation
  pyplot.plot(x,y,"k-", label="network")
  pyplot.legend()

### Task 9

For each of the datasets and their according optimized parameters, call the plotting function from Task 8. Use range $R=[-3,3]$ for dataset $X_1$ and $X_2$, and range $R=[-5.5,4.5]$ for dataset $X_3$. Note that the first element of range $R$ should be the lowest $x$-location, and the second element of $R$ the highest value for $x$. Did the networks approximate the functions? What can we do if not?

In [ ]:
figure = pyplot.figure(figsize=(10,3))

# plot first function
pyplot.subplot(131)
...

# plot second function
pyplot.subplot(132)
...

# plot third function
pyplot.subplot(133)
...